In [1]:

"""
Function defination: load CNN parameters and INFMAP with data quantization

"""

import numpy as np
def load_weight(file_path, in_channels, kernel_num, kernel_dim):
    weight_matrix = np.zeros((in_channels, kernel_num, kernel_dim, kernel_dim))
    data = []
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            value =int( round(float(line.strip().rstrip(','))*2**15))
            data.append(value)
    index = 0
    
    print(file_path, max(data),min(data))
    
    for m in range (in_channels):
        for k in range(kernel_num):
            for i in range(kernel_dim):
                for j in range(kernel_dim):
                    weight_matrix[m, k, i, j] = data[index]
                    index += 1
    return weight_matrix

def load_weight_fc(file_path,  kernel_num, kernel_element):
    weight_matrix = np.zeros(( kernel_num,kernel_element))
    data = []
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            value =int( round(float(line.strip().rstrip(','))*2**15))
            data.append(value)
    index = 0
    
    print(file_path, max(data),min(data))
    
    for i in range(kernel_num):
        for j in range(kernel_element):
                weight_matrix[ i, j] = data[index]
                index += 1
    return weight_matrix

def load_bias(file_path,  kernel_num):
    bias_matrix = np.zeros(( kernel_num))
    data = []
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            value = int( round(float(line.strip().rstrip(','))*2**15))
            data.append(value)
    index = 0
    
    print(file_path, max(data),min(data))
    
    for i in range(kernel_num):
            bias_matrix[ i] = data[index]
            index += 1
    return bias_matrix
             
def load_data(file_path, infmap_rows, infmap_cols):
    data_matrix = np.zeros((infmap_rows,infmap_cols))
    data = []
    index = 0
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            value = int(line.strip().rstrip(','))
            data.append(value)
   
    for i in range(infmap_rows):
        for j in range(infmap_cols):   
                data_matrix[ i, j] = data[index]
                index += 1
    return data_matrix

In [2]:
"""
Using Lenet model to infer images with data quantization (this can show the intermediate reults of each layer)

"""

import numpy as np
import torch
import torch.nn.functional as F

# 参数定义
INFMAP_ROWS = 32
INFMAP_COLS =32

IN_CHANNEL1 =1
KERNEL_NUM1 = 6
KERNEL_DIM1= 5


IN_CHANNEL2 =6
KERNEL_NUM2 = 16
KERNEL_DIM2= 5

IN_CHANNEL3 =16
KERNEL_NUM3 = 120
KERNEL_DIM3= 5


IN_CHANNEL4 =1
KERNEL_NUM4 = 84
KERNEL_ELEMENT4= 120


IN_CHANNEL5 =1
KERNEL_NUM5 = 10
KERNEL_ELEMENT5= 84
data_path = './cnn_parameter/infmap_binary.txt'
weight_path1 = './cnn_parameter/conv1.weight.txt'
weight_path2 = './cnn_parameter/conv2.weight.txt'
weight_path3 = './cnn_parameter/conv3.weight.txt'
weight_path4 = './cnn_parameter/fc1.weight.txt'
weight_path5 = './cnn_parameter/fc2.weight.txt'

bias_path1 = './cnn_parameter/conv1.bias.txt'
bias_path2 = './cnn_parameter/conv2.bias.txt'
bias_path3 = './cnn_parameter/conv3.bias.txt'
bias_path4 = './cnn_parameter/fc1.bias.txt'
bias_path5 = './cnn_parameter/fc2.bias.txt'

data = load_data(data_path,INFMAP_ROWS,INFMAP_COLS)
weight1= load_weight(weight_path1,IN_CHANNEL1,KERNEL_NUM1,KERNEL_DIM1)
#rint(weight1)
weight2= load_weight(weight_path2,IN_CHANNEL2,KERNEL_NUM2,KERNEL_DIM2)
weight3= load_weight(weight_path3,IN_CHANNEL3,KERNEL_NUM3,KERNEL_DIM3)

weight4= load_weight_fc(weight_path4,KERNEL_NUM4,KERNEL_ELEMENT4)
weight5=load_weight_fc(weight_path5,KERNEL_NUM5,KERNEL_ELEMENT5)


bias1 = load_bias(bias_path1,KERNEL_NUM1)
bias2 = load_bias(bias_path2,KERNEL_NUM2)
bias3 = load_bias(bias_path3,KERNEL_NUM3)
bias4 = load_bias(bias_path4,KERNEL_NUM4)
bias5 =load_bias(bias_path5,KERNEL_NUM5)
#rint(bias5)
data_tensor = torch.tensor(data, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
weight1_tensor = torch.tensor(weight1, dtype=torch.float32).view(KERNEL_NUM1, IN_CHANNEL1, KERNEL_DIM1, KERNEL_DIM1)
bias1_tensor = torch.tensor(bias1, dtype=torch.float32)



weight2_tensor = torch.tensor(weight2, dtype=torch.float32).view(KERNEL_NUM2, IN_CHANNEL2, KERNEL_DIM2, KERNEL_DIM2)
bias2_tensor = torch.tensor(bias2, dtype=torch.float32)


weight3_tensor = torch.tensor(weight3, dtype=torch.float32).view(KERNEL_NUM3, IN_CHANNEL3, KERNEL_DIM3, KERNEL_DIM3)
bias3_tensor = torch.tensor(bias3, dtype=torch.float32)

weight4_tensor = torch.tensor(weight4, dtype=torch.float32)
bias4_tensor = torch.tensor(bias4, dtype=torch.float32)

weight5_tensor = torch.tensor(weight5, dtype=torch.float32)
bias5_tensor = torch.tensor(bias5, dtype=torch.float32)

torch.set_printoptions(edgeitems=32, linewidth=1000, precision=2)

np.set_printoptions(suppress=True)


#layer1
conv1 = F.conv2d(data_tensor, weight1_tensor) + bias1_tensor.view(1, KERNEL_NUM1, 1, 1)
pool1 = F.max_pool2d(conv1, kernel_size=2, stride=2)
o1 = torch.floor(pool1/(2**15))
relu1 = F.relu(o1)
print("layer1_conv:  \n", conv1) 
print("layer1_pool：  \n",pool1)
print("layer1_relu:  \n",relu1)


#layer2
conv2 = F.conv2d(relu1, weight2_tensor) + bias2_tensor.view(1, KERNEL_NUM2, 1, 1)
pool2 = F.max_pool2d(conv2, kernel_size=2, stride=2)
o2 = torch.floor(pool2/(2**15))
relu2 = F.relu(o2)
print("layer2_conv: \n", conv2) 
print("layer2_pool：  \n",pool2)
print("layer2_relu ： \n",relu2)



#layer3
conv3 = F.conv2d(relu2, weight3_tensor)+ bias3_tensor.view(1, KERNEL_NUM3, 1, 1)
o3 = torch.floor(conv3/(2**15))
relu3 = F.relu(o3)
print("layer3_conv:   \n", conv3) 
print("layer3_relu  \n",relu3)


#fc1
flatten = relu3.view(-1, 120)
fc1 = F.linear(flatten, weight4_tensor)+bias4_tensor
o4= torch.floor(fc1/(2**15))
relu4 = F.relu(o4)
print("fc1_fc: \n ", fc1) 
print("fc1_relu: \n ", relu4)

flatten1 = relu4.view(-1, 84)
fc2 = F.linear(flatten1, weight5_tensor)+bias5_tensor
o5= torch.floor(fc2/(2**15))
print("fc2_fc: \n ", o5)



            
               
               



./cnn_parameter/conv1.weight.txt 27883 -22781
./cnn_parameter/conv2.weight.txt 28409 -13450
./cnn_parameter/conv3.weight.txt 8002 -8872
./cnn_parameter/fc1.weight.txt 9069 -9351
./cnn_parameter/fc2.weight.txt 17114 -16428
./cnn_parameter/conv1.bias.txt 10587 -10342
./cnn_parameter/conv2.bias.txt 2805 -5489
./cnn_parameter/conv3.bias.txt 2412 -2011
./cnn_parameter/fc1.bias.txt 4930 -3566
./cnn_parameter/fc2.bias.txt 3198 -5556
layer1_conv:  
 tensor([[[[ 1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03],
          [ 1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.16e+03,  1.

  tensor([[ 182523.,  -23676.,  -20910., -127674.,  -56432.,  -50801.,  -35892.,  -94429.,   42995.,   -4018.,  -10396.,    5696.,   39427.,   51834.,  113464.,  -37810.,   -3961., -117542.,   -6603.,   -3316.,  -42766., -148463.,  -27384.,   -8206., -122147.,   18404.,   63719.,   -2309.,  -52539.,    6933.,   91845.,  -21436.,  106339.,   34864.,  221225.,  136710.,   23282.,  -21322.,   -9631.,    7827.,  -62650.,  -28824.,  184114.,   18348.,   53564., -105501.,   70995.,   18618.,  -65734.,  -23374.,   55203.,   89620.,  -64870.,  -46587.,   47290.,   73892.,  -71360.,    1324.,  -65390.,   93768.,  151820.,  -29942.,   12790.,    4083., -125768.,  -34814.,   -2686.,   43190.,  -81901.,  -26364.,   68841.,   18531.,  -50297.,   84094.,  -43608.,   38204.,   19364.,  -78165., -128184.,   37182.,   64680.,  -41731.,   77009.,  -34723.]])
fc1_relu: 
  tensor([[5., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 2., 0

In [3]:

"""
Generate coe files which are used to initialize the rom in the FPGA

"""
import numpy as np
import torch



def generate_data_coe(data):
   
    num_rows = data.shape[0]
    for r in range(num_rows):
        data_row = data[r, :]  
        filename = f"./coe/data_row{r}.coe"
        write_coe_file(filename, data_row)
        print(f"Generated {filename}")

def write_coe_file(filename, data, radix=10):
   
    with open(filename, 'w') as f:
        f.write(f"memory_initialization_radix={radix};\n")
        f.write("memory_initialization_vector=\n")
        if radix == 16:
            data_str = [format(int(x), '02X') for x in data.flatten()]  
        else:
            data_str = [str(int(x)) for x in data.flatten()] 
        f.write(",\n".join(data_str) + ";\n")


def generate_weight_coe(weights, prefix):
    num_kernels, in_channels, kernel_dim, _  = weights.shape
    coe_files = {}
    for k in range(num_kernels):
        combined_weight = weights[k].flatten()
        group = k % 4
        if group not in coe_files:
            coe_files[group] = []
        coe_files[group].append(combined_weight)

    for group, data_list in coe_files.items():
        combined_data = np.concatenate(data_list)
        filename = f"./coe/{prefix}_kernels_group{group}.coe"
        write_coe_file(filename, combined_data)
        print(f"Generated {filename}")
        
        
def generate_weight_fc_coe(weights, prefix):
    num_kernels, _ = weights.shape
 
    coe_files = {}
    for k in range(num_kernels):
        
        combined_weight = weights[k].flatten()
        group = k % 4
        if group not in coe_files:
            coe_files[group] = []
        coe_files[group].append(combined_weight)

    for group, data_list in coe_files.items():
        combined_data = np.concatenate(data_list)
        filename = f"./coe/{prefix}_kernels_group{group}.coe"
        write_coe_file(filename, combined_data)
        print(f"Generated {filename}")
        
        
def generate_bias_coe(bias, prefix):
    num_kernels = bias.shape[0]
 
    coe_files = {}
    for k in range(num_kernels):
        combined_bias = bias[k].flatten()
        group = k % 4
        if group not in coe_files:
            coe_files[group] = []
        coe_files[group].append(combined_bias)

    for group, data_list in coe_files.items():
        
        combined_data = np.concatenate(data_list)
        filename = f"./coe/{prefix}_bias_group{group}.coe"
        write_coe_file(filename, combined_data)
        print(f"Generated {filename}")

in_data_mem_2d = data_tensor.view(32, 32).to(torch.int32).numpy()
in_weight_mem1_int = weight1_tensor.to(torch.int32).numpy()
in_weight_mem2_int = weight2_tensor.to(torch.int32).numpy()
in_weight_mem3_int = weight3_tensor.to(torch.int32).numpy()
in_weight_mem4_int = weight4_tensor.to(torch.int32).numpy()
in_weight_mem5_int = weight5_tensor.to(torch.int32).numpy()

in_bias_mem1_int = bias1_tensor.to(torch.int32).numpy()
in_bias_mem2_int = bias2_tensor.to(torch.int32).numpy()
in_bias_mem3_int = bias3_tensor.to(torch.int32).numpy()
in_bias_mem4_int = bias4_tensor.to(torch.int32).numpy()
in_bias_mem5_int = bias5_tensor.to(torch.int32).numpy()



generate_data_coe(in_data_mem_2d)

generate_weight_coe(in_weight_mem1_int, "weight1")
generate_weight_coe(in_weight_mem2_int, "weight2")
generate_weight_coe(in_weight_mem3_int, "weight3")


generate_weight_fc_coe(in_weight_mem4_int, "weight4")
generate_weight_fc_coe(in_weight_mem5_int, "weight5")

generate_bias_coe(in_bias_mem1_int, "bias1")
generate_bias_coe(in_bias_mem2_int, "bias2")
generate_bias_coe(in_bias_mem3_int, "bias3")
generate_bias_coe(in_bias_mem4_int, "bias4")
generate_bias_coe(in_bias_mem5_int, "bias5")

Generated ./coe/data_row0.coe
Generated ./coe/data_row1.coe
Generated ./coe/data_row2.coe
Generated ./coe/data_row3.coe
Generated ./coe/data_row4.coe
Generated ./coe/data_row5.coe
Generated ./coe/data_row6.coe
Generated ./coe/data_row7.coe
Generated ./coe/data_row8.coe
Generated ./coe/data_row9.coe
Generated ./coe/data_row10.coe
Generated ./coe/data_row11.coe
Generated ./coe/data_row12.coe
Generated ./coe/data_row13.coe
Generated ./coe/data_row14.coe
Generated ./coe/data_row15.coe
Generated ./coe/data_row16.coe
Generated ./coe/data_row17.coe
Generated ./coe/data_row18.coe
Generated ./coe/data_row19.coe
Generated ./coe/data_row20.coe
Generated ./coe/data_row21.coe
Generated ./coe/data_row22.coe
Generated ./coe/data_row23.coe
Generated ./coe/data_row24.coe
Generated ./coe/data_row25.coe
Generated ./coe/data_row26.coe
Generated ./coe/data_row27.coe
Generated ./coe/data_row28.coe
Generated ./coe/data_row29.coe
Generated ./coe/data_row30.coe
Generated ./coe/data_row31.coe
Generated ./coe/we

In [5]:
def load_data(file_path, infmap_rows, infmap_cols, image_index):
    data_matrix = np.zeros((infmap_rows,infmap_cols*image_index))
    data = []
    index = 0
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            value = int(line.strip().rstrip(','))
            data.append(value)
            
    for a in range (image_index):
        for i in range(infmap_rows):
            for j in range(infmap_cols):   
                    data_matrix[ i, j+a*infmap_cols] = data[a*infmap_cols*infmap_rows+i*infmap_cols+j]
                    
    return data_matrix

def generate_data_coe(data):
   
    num_rows = data.shape[0]
    for r in range(num_rows):
        data_row = data[r, :]  
        filename = f"./coe/mul_data_row{r}.coe"
        write_coe_file(filename, data_row)
        print(f"Generated {filename}")


data_path1 = './cnn_parameter/infmap_binary_mul.txt'
data1 = load_data(data_path1,32,32,12)
data_tensor1 = torch.tensor(data1, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
in_data_mem_2d1 = data_tensor1.view(32, 384).to(torch.int32).numpy()
generate_data_coe(in_data_mem_2d1)

Generated ./coe/mul_data_row0.coe
Generated ./coe/mul_data_row1.coe
Generated ./coe/mul_data_row2.coe
Generated ./coe/mul_data_row3.coe
Generated ./coe/mul_data_row4.coe
Generated ./coe/mul_data_row5.coe
Generated ./coe/mul_data_row6.coe
Generated ./coe/mul_data_row7.coe
Generated ./coe/mul_data_row8.coe
Generated ./coe/mul_data_row9.coe
Generated ./coe/mul_data_row10.coe
Generated ./coe/mul_data_row11.coe
Generated ./coe/mul_data_row12.coe
Generated ./coe/mul_data_row13.coe
Generated ./coe/mul_data_row14.coe
Generated ./coe/mul_data_row15.coe
Generated ./coe/mul_data_row16.coe
Generated ./coe/mul_data_row17.coe
Generated ./coe/mul_data_row18.coe
Generated ./coe/mul_data_row19.coe
Generated ./coe/mul_data_row20.coe
Generated ./coe/mul_data_row21.coe
Generated ./coe/mul_data_row22.coe
Generated ./coe/mul_data_row23.coe
Generated ./coe/mul_data_row24.coe
Generated ./coe/mul_data_row25.coe
Generated ./coe/mul_data_row26.coe
Generated ./coe/mul_data_row27.coe
Generated ./coe/mul_data_row28